In [1]:
# 2019 Q3
# In 2019Q3, checked on LR Connection UI, only quarterly segments & new sign ups
import pandas as pd
import numpy as np
import datetime
import os
import gc
import glob
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2019_Q3'

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [3]:
'''
Segments:
1. Quarterly regular
2. New SignUps
'''

'\nSegments:\n1. Quarterly regular\n2. New SignUps\n'

In [4]:
seg_1=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/upload_files/"))
print("seg_1: "+str(len(seg_1)))

seg_2=list(recursive_file_gen("/home/jian/celery/BiWeekly_New_SignUps/monthly_update/"))
seg_2=[x for x in seg_2 if ".csv" in x]
seg_2=pd.DataFrame({"file_path":seg_2})
seg_2['date']=seg_2['file_path'].apply(lambda x: x.split("onthly_update/output_")[1][:10])
seg_2=seg_2[seg_2['date']>="2019-07-27"] # 1 wave ahead of the quarter
seg_2=seg_2[seg_2['date']<"2019-11-02"] # 
seg_2=seg_2['file_path'].tolist()
print("seg_2: "+str(len(seg_2)))

seg_1: 50
seg_2: 18


In [5]:
all_file_list=seg_1+seg_2
len(all_file_list)

68

In [6]:
list_all_Q3_seg=[]
col_list=[]
for file in all_file_list:
    try:
        df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','email_address_hash','segment'])
    except:
        df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','Email_Address_hash','segment'])
        df=df.rename(columns={"Email_Address_hash":"email_address_hash"})
        
    list_all_Q3_seg.append(df)
df_output_all_Q3_seg=pd.concat(list_all_Q3_seg)
del list_all_Q3_seg

print(df_output_all_Q3_seg.shape,df_output_all_Q3_seg['email_address_hash'].nunique())

(24078232, 3) 24078139


In [7]:
local_path_agg_dup="/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2019_Q3/BL_2019Q3_all_segments_aggregated_JL_"+str(datetime.datetime.now().date())+".csv"

df_output_all_Q3_seg.to_csv(local_path_agg_dup,index=False)

In [8]:
df_output_unique_email_Q3_mapping=df_output_all_Q3_seg.drop_duplicates("email_address_hash")
print(df_output_unique_email_Q3_mapping.shape,df_output_unique_email_Q3_mapping['email_address_hash'].nunique())
local_mapping="/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2019_Q3/BL_mapping_file_2019Q3_JL_"+str(datetime.datetime.now().date())+".csv"

df_output_unique_email_Q3_mapping.to_csv(local_mapping,index=False)

(24078139, 3) 24078139


In [9]:
import paramiko

host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)


In [10]:
print("local_mapping: ",local_mapping)
print("local_path_agg_dup: ",local_path_agg_dup)

local_mapping:  /home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2019_Q3/BL_mapping_file_2019Q3_JL_2019-11-20.csv
local_path_agg_dup:  /home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2019_Q3/BL_2019Q3_all_segments_aggregated_JL_2019-11-20.csv


In [11]:
print(datetime.datetime.now())

2019-11-20 16:59:02.730595


In [12]:
# local_mapping
remote_folder='/mnt/drv5/jian/Big_Lots/BL_1_to_1_2019Q3/Q3_mapping_file/'
basename=os.path.basename(local_mapping)
remote_path=remote_folder+basename
sftp.put(local_mapping,remote_path)

<SFTPAttributes: [ size=3772960887 uid=518 gid=551 mode=0o100644 atime=1574287140 mtime=1574287474 ]>

In [13]:
# local_path_agg_dup
remote_folder='/mnt/drv5/jian/Big_Lots/BL_1_to_1_2019Q3/Aggregated_all_2019Q3_LR_segments/'
basename=os.path.basename(local_path_agg_dup)
remote_path=remote_folder+basename
sftp.put(local_path_agg_dup,remote_path)

<SFTPAttributes: [ size=3772974744 uid=518 gid=551 mode=0o100644 atime=1574281904 mtime=1574287816 ]>

In [14]:
sftp.close()
transport.close()

print("done: ",datetime.datetime.now())

done:  2019-11-20 17:10:19.696711
